# Functionality

Simulated data for validation the usage of ciCOH

* using Samplings from normal distributions for no connections

* using sinusoidal signals for connections

* using a linear mix equation for volume conduction

## Import Packages

In [1]:
import os, sys
import scipy.io as sio
import glob
import numpy as np
import matplotlib.pyplot as plt
import pickle 
import random
import pandas as pd
%matplotlib inline
plt.rc('text', usetex=True)

from scipy.stats import norm
from numpy.random import normal

In [2]:
# extract the exp folder path
currfolder = os.getcwd()
codefolder = currfolder[0 : currfolder.find('code')+len('code')]

# add path the exp folder
sys.path.append(codefolder)

# import_nbmodule used for import package in .ipynb
import import_nbmodule

# import util/folder_extract.pynb 
from util.folder_extract import exp_subfolders, code_corresfolder

# import ciCoherence_overtime in connAnalyTool/synchronization_indices.ipynb
from connAnalyTool.synchronization_indices import ciCoherence_acrosstrials
from connAnalyTool.synchronization_indices import ciCoherence_overtime

from simulated.simulated_timeseries import gen_series_noconnection_sin, gen_ciCOH_population_sin

importing Jupyter notebook from /home/lingling/Insync/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/simulated/simulated_timeseries.ipynb
importing Jupyter notebook from /home/lingling/Insync/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/simulated/addnoise.ipynb


## Preparation

### exp subfolders & code_corresfolder

In [3]:
_, _, pipelinefolder, _= exp_subfolders()

In [4]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [5]:
nb_name = nb_name[0: nb_name.find('.ipynb')]

# corresfolder
corresfolder, correparentfolder = code_corresfolder(os.getcwd(), nb_name)

### Global parameters

In [6]:
ntemp, ntrials = 500, 200

# repeat ntimes
ntimes = 1000

f = 30


t = 1


Desired_SNR_dB = 20

### Save setup

In [ ]:
savefolder = corresfolder

## No Connection

### ciCOH

In [ ]:
# generate the ciCOH population 
ciCOHs_noConn = gen_ciCOH_population_sin(ntimes = ntimes, ntrials = ntrials, ntemp = ntemp, f =f, t = t, Desired_SNR_dB = 20)

In [ ]:
#---- plot ciCOHs_noConn  ----#
ciCOHs = ciCOHs_noConn

plt.hist(ciCOHs, bins=10, density=True, alpha=0.6, color='g')

# Fit a normal distribution to the data:
mu, std = norm.fit(ciCOHs)

# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)

# lot the 2*std lines
plt.plot([mu + 2 * std, mu + 2 * std], np.array(plt.ylim())/2, 'b--') 
plt.plot([mu - 2 * std, mu - 2 * std], np.array(plt.ylim())/2, 'b--', label = '95\%')

# plot the 3*std lines
plt.plot([mu + 3 * std, mu + 3 * std], np.array(plt.ylim())/2, 'k--') 
plt.plot([mu - 3 * std, mu - 3 * std], np.array(plt.ylim())/2, 'k--', label = r'99\%')

plt.legend(ncol=2)




# Plot statistical analysis:  mu, std and 95% 

# the content of statistical analysis text
eq1 = r"\begin{eqnarray*}" + \
  r"\mu = " + str(np.around(mu, decimals=2)) + "\\"\
  r"\std = " + str(np.around(std, decimals=4)) + "\\"\
  r"\95 \%= " + str(np.around(mu + 2*std, decimals=4)) + "\\"\
  r"\99 \%= " + str(np.around(mu + 3*std, decimals=4)) +\
  r"\end{eqnarray*}"


# the plot position of mu, std, and 95% text
ymin, ymax = plt.ylim()
xmin, xmax = plt.xlim()
pos_x = xmin + (xmax - xmin)* 3/4
pos_y = ymin + (ymax -ymin) * 4/5
plt.text(pos_x, pos_y, eq1, {'fontsize': 12}, va="top", ha="left")


#plt.show()
plt.savefig(os.path.join(savefolder,  "sin_noConn_ciCOHsDistribution.png"))

del ciCOHs
del mu, std, xmin, xmax, p
del eq1
del ymin, ymax, pos_x, pos_y

### A trial Example

In [ ]:
# generate 2 sinusoidal time series without connection sig1_noConn, sig2_noConn: ntrials * ntemp
sig1_noConn, sig2_noConn = gen_series_noconnection_sin(ntrials, ntemp, f, t = t, Desired_SNR_dB = 20)

In [ ]:
#---- plot one trial Example ---#
npoints = 125
ts = np.linspace(0, t, ntemp)


fig, ax = plt.subplots()

plt.subplot(211)

plt.plot(ts[0:npoints], sig1_noConn[0, 0:npoints], 'k', label = 'signal 1') 
plt.plot(ts[0:npoints], sig2_noConn[0, 0:npoints], 'b', label = 'signal 2')

title = "An example of time series without connections"
plt.title(title, size=15)

bottom, top = plt.ylim()
plt.ylim([bottom, top + (top - bottom)* 0.2])

plt.ylabel('trial 1')
plt.legend(ncol = 2, loc = 1)

plt.gca().set_xticklabels([])


plt.subplot(212)
plt.plot(ts[0:npoints], sig1_noConn[1, 0:npoints], 'k', label = 'signal 1') 
plt.plot(ts[0:npoints], sig2_noConn[1, 0:npoints], 'b', label = 'signal 2')

plt.xlabel('time (s)', size=15)
plt.axis('tight')
plt.ylabel('trial 2')



#plt.show()
plt.savefig(os.path.join(savefolder,  "sin_noConn_Example.png.png"))

## with Connection

In [ ]:
### functioon add Gaussian Noise
def GaussianNoise_SNR(signal, Desired_SNR_dB):
    """
        add desired SNR (dB) gaussian noise to signal
        SNR(dB) = 10 * log10(power_signal/power_noise)

        @ parameter:
            signal: (n_times,)
            Desired_SNR_dB: desired SNR in dB
        
        @ return 
            signal_noisy: (n_times,)
    """
    
    n_times = signal.shape[0]
    noise = np.random.normal(loc=0.0, scale=1.0, size=(n_times,))

    power_signal = np.dot(abs(signal), abs(signal))/n_times
    power_noise = np.dot(abs(noise), abs(noise))/n_times 

    k = (power_signal * pow(10,(-Desired_SNR_dB/10)))/power_noise # scale factor
    noise_new = np.sqrt(k) * noise

    signal_noisy = signal + noise_new

    return signal_noisy

### Gen series 

In [ ]:
ts = np.linspace(0,t, ntemp)


# generated s1, s2 and s3 signals: ntimes * ntrials * ntemp
sigs1, sigs2 = np.empty(shape=(0, ntrials, ntemp)), np.empty(shape=(0, ntrials, ntemp))
sigs3 = np.empty(shape=(0, ntrials, ntemp))
for timei in range(ntimes):
    
    if timei % 100 ==0:
        print("timei = " + str(timei) + "/" + str(ntimes))
    
    # generated s1, s2 and s3 signals: ntrials * ntemp
    sig1, sig2, sig3 = np.empty(shape=[0,ntemp]), np.empty(shape=[0,ntemp]), np.empty(shape=[0,ntemp])
    phi1 = normal(loc=np.pi, scale=1.0)
    phi2 = normal(loc=np.pi, scale=1.0)
    for triali in range(ntrials):
        
        s1 = np.sin(2 * np.pi * f * ts)
        s2 = np.sin(2 * np.pi * f * ts + phi1)
        s3 = 2 * np.sin(2 * np.pi * f * ts + phi2)

        # add normal distribution noise
        s1 = GaussianNoise_SNR(s1, Desired_SNR_dB = Desired_SNR_dB)
        s2 = GaussianNoise_SNR(s2, Desired_SNR_dB = Desired_SNR_dB)
        s3 = GaussianNoise_SNR(s3, Desired_SNR_dB = Desired_SNR_dB)

        # append the time serie of the new trial
        sig1 = np.append(sig1, np.expand_dims(s1, axis = 0), axis=0)
        sig2 = np.append(sig2, np.expand_dims(s2, axis = 0), axis=0)
        sig3 = np.append(sig3, np.expand_dims(s3, axis = 0), axis=0)
        
        del s1, s2, s3
        
    # append the time serie of the new trial
    sigs1 = np.append(sigs1, np.expand_dims(sig1, axis = 0), axis=0)
    sigs2 = np.append(sigs2, np.expand_dims(sig2, axis = 0), axis=0)
    sigs3 = np.append(sigs3, np.expand_dims(sig3, axis = 0), axis=0)
    
    del sig1, sig2, sig3
    
sigs1_conn, sigs2_conn, sigs3_conn = sigs1, sigs2, sigs3

del sigs1, sigs2, sigs3, ts

###  An Example

In [ ]:
sigs1_conn.shape

#---- plot one trial Example ---#
npoints = 125
ts = np.linspace(0, t, ntemp)

timei = 0

fig, ax = plt.subplots()


triali = 0
plt.subplot(211)
plt.plot(ts[0:npoints], sigs1_conn[timei, triali, 0:npoints], 'k', label = 'signal 1') 
plt.plot(ts[0:npoints], sigs2_conn[timei, triali, 0:npoints], 'b', label = 'signal 2')
plt.plot(ts[0:npoints], sigs3_conn[timei, triali, 0:npoints], 'r', label = 'signal 3')

title = "An example of time series without connections"
plt.title(title, size=15)

bottom, top = plt.ylim()
plt.ylim([bottom, top + (top - bottom)* 0.2])

plt.ylabel('trial 1')
plt.legend(ncol = 3, loc = 1)

plt.gca().set_xticklabels([])


triali = 1
plt.subplot(212)
plt.plot(ts[0:npoints], sigs1_conn[timei, triali, 0:npoints], 'k', label = 'signal 1') 
plt.plot(ts[0:npoints], sigs2_conn[timei, triali, 0:npoints], 'b', label = 'signal 2')
plt.plot(ts[0:npoints], sigs3_conn[timei, triali, 0:npoints], 'r', label = 'signal 3')

plt.xlabel('time (s)', size=15)
plt.axis('tight')
plt.ylabel('trial 2')



#plt.show()
plt.savefig(os.path.join(savefolder,  "sin_conn_Example.png.png"))

del npoints, ts, fig, ax, triali, timei

### ciCOH 

In [ ]:
#-- calc ciCOH for conn ---#
sigs1, sigs2, sigs3 = sigs1_conn, sigs2_conn, sigs3_conn 

ciCOHs_12, ciCOHs_13 = [], []
for timei in range(ntimes):
    
    if timei % 100 ==0:
        print("timei = " + str(timei) + "/" + str(ntimes))
    
    sig1, sig2, sig3 = sigs1[timei, :, :], sigs2[timei, :, :], sigs3[timei, :, :]
        
    # calculate ciCOH
    ciCOH_12 = np.mean(ciCoherence_acrosstrials(sig1, sig2))
    ciCOH_13 = np.mean(ciCoherence_acrosstrials(sig1, sig3))
    ciCOHs_12.append(ciCOH_12)
    ciCOHs_13.append(ciCOH_13)
        
    
    del sig1, sig2, sig3, ciCOH_12, ciCOH_13
    
ciCOHs_conn_sin12, ciCOHs_conn_sin13 = ciCOHs_12, ciCOHs_13
del ciCOHs_12, ciCOHs_13, sigs1, sigs2, sigs3

### plot ciCOH

In [ ]:
#--- plot ciCOH_conn --- # 

alpha1 = 0.05
alpha2 = 0.01


plt.figure()


### ciCOHs_12

ciCOHs = ciCOHs_conn_sin12
ciCOHs = abs(np.array(ciCOHs))

# P(ciCOH >v_thred) = 1- alpha
v1_thred = np.percentile(abs(ciCOHs), alpha1 * 100)
v2_thred = np.percentile(abs(ciCOHs), alpha2 * 100)

# Fit a normal distribution to the data:
mu, std = norm.fit(abs(ciCOHs))

# Plot the histogram 
plt.subplot(211)
plt.hist(ciCOHs, bins=25, density = True, alpha=0.6, color='g')

eq1 = r"\begin{eqnarray*}" + \
      r"\\95 \% > " + str(np.around(v1_thred, decimals=3)) + \
      r"\\99 \% > " + str(np.around(v2_thred, decimals=3)) + \
      r"\end{eqnarray*}"

bottom, top = plt.ylim()
plt.text(0.2, bottom + (top - bottom) * 0.5, eq1, {'fontsize': 12}, va="top", ha="left")
plt.text(0.4, bottom + (top - bottom) * 0.9, r"s1 and s2", {'fontsize': 12}, va="top", size=12, weight="bold")
plt.xticks([])


title = "Histogram of abs(ciCOH) data"
plt.title(title, size=15)

del ciCOHs, mu, std, v1_thred, v2_thred, eq1, bottom, top, title



### ciCOHs_13

ciCOHs = ciCOHs_conn_sin13
ciCOHs = abs(np.array(ciCOHs))

# P(ciCOH >v_thred) = 1- alpha
v1_thred = np.percentile(abs(ciCOHs), alpha1 * 100)
v2_thred = np.percentile(abs(ciCOHs), alpha2 * 100)

# Fit a normal distribution to the data:
mu, std = norm.fit(abs(ciCOHs))

plt.subplot(212)
plt.hist(ciCOHs, bins=25, density = True, color = 'g', alpha = 0.6)


eq1 = r"\begin{eqnarray*}" + \
      r"\\95 \% > " + str(np.around(v1_thred, decimals=3)) + \
      r"\\99 \% > " + str(np.around(v2_thred, decimals=3)) + \
      r"\end{eqnarray*}"


bottom, top = plt.ylim()


plt.text(0.2, bottom + (top - bottom) * 0.5, eq1, {'fontsize': 12}, va="top", ha="left")
plt.text(0.4, bottom + (top - bottom) * 0.9, r"s1 and s3", {'fontsize': 12}, va="top", size=12, weight="bold")


plt.xlabel('ciCOH Values', size=15)
plt.text(-0.18, bottom + (top - bottom) * 0.7,'Probability Density', size=15, rotation='vertical')

#plt.show()
plt.savefig(os.path.join(savefolder,  "sin_conn_ciCOHDistribution.png"))

del ciCOHs, mu, std, v1_thred, v2_thred, eq1, bottom, top

## Volume Conduction

### no Conn

In [ ]:
sigs1, sigs2 = np.zeros((ntimes, ntrials, ntemp)), np.empty((ntimes, ntrials, ntemp))

for timei in range(ntimes):

    if timei  % 100 ==0:
        print("run the sinc simulation at timei = " + str(timei) + "/" + str(ntimes))

    # generate 2 sinusoidal time series without connection sig1_noConn, sig2_noConn: ntrials * ntemp
    sig1, sig2 = gen_series_noconnection_sin(ntrials = ntrials, ntemp = ntemp, f = f, t = t, 
                                         Desired_SNR_dB = Desired_SNR_dB)
    
    sigs1[timei, :, :] = sig1
    sigs2[timei, :, :] = sig2
    
    del sig1, sig2

sigs1_noconn, sigs2_noconn = sigs1, sigs2
del sigs1, sigs2

In [ ]:
alpha = 0.9
sigs1_new = sigs1_noconn  + sigs2_noconn * alpha
sigs2_new = sigs2_noconn  + sigs1_noconn * alpha

In [ ]:
x, y = x_noConn[timei, :, :], y_noConn[timei, :, :] 

x_new, y_new = x[:, 201:500] + alpha * y[:, 1:300],  y[:, 201:500] + alpha * x{:, 1:300}

x.shape

In [ ]:
### no connection ####

x_noConn, y_noConn = sigs1_noconn, sigs2_noconn
ciCOH_ori, ciCOH_new = [], []
for timei in range(x_noConn.shape[0]):

    x, y = x_noConn[timei, :, :], y_noConn[timei, :, :] 
    x_new, y_new = x[:, 201:500] + alpha * y[:, 1:300],  y[:, 201:500] + alpha * x[:, 1:300]
    x, y = x[:, 201:500], y[:, 201:500]
    
    # calculate ciCOH
    ciCOH_ori = np.append(ciCOH_ori, np.mean(ciCoherence_acrosstrials(x, y)))
    ciCOH_new = np.append(ciCOH_new, np.mean(ciCoherence_acrosstrials(x_new, y_new)))

    del x,  y, x_new, y_new
    
ciCOH_noConn_ori, ciCOH_noConn_new = ciCOH_ori, ciCOH_new
del ciCOH_ori, ciCOH_new

In [ ]:
from scipy.stats import norm
import matplotlib.pyplot as plt
plt.rc('text', usetex=False)

fig, ax = plt.subplots()
plt.plot(np.abs(ciCOH_noConn_ori), np.abs(ciCOH_noConn_new), '.')


ax.set(xlabel='x and y', ylabel='xnew and ynew',
       title='ciCOH values')

plt.savefig(os.path.join(savefolder,  "volume_noconn.png"))

### Conn

In [ ]:
### connection ####

x_Conn, y_Conn = sigs1_sin, sigs2_sin
ciCOH_ori, ciCOH_new = [], []
for timei in range(x_Conn.shape[0]):

    x, y = x_Conn[timei, :, :], y_Conn[timei, :, :] 
    x_new, y_new = x + alpha * y,  y + alpha * x

    # calculate ciCOH
    ciCOH_ori = np.append(ciCOH_ori, np.mean(ciCoherence_acrosstrials(x, y)))
    ciCOH_new = np.append(ciCOH_new, np.mean(ciCoherence_acrosstrials(x_new, y_new)))

    del x,  y, x_new, y_new
    
ciCOH_Conn_ori, ciCOH_Conn_new = ciCOH_ori, ciCOH_new
del ciCOH_ori, ciCOH_new

### plot

In [ ]:
ax

In [ ]:
from scipy.stats import norm
import matplotlib.pyplot as plt

plt.plot(ciCOH_Conn_ori, ciCOH_Conn_new, '.')
plt.xlabel('ciCOH values between two time series with no connections')
plt.ylabel('ciCOH values between two new generated time series')

plt.savefig(os.path.join(savefolder,  "volume_conn.png"))